In [1]:
import json
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.types import StructType

In [2]:
# Criando SparkSession
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .config("spark.mongodb.output.uri", "mongodb://127.0.0.1/") \
        .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1')\
        .getOrCreate()

21/11/11 19:51:31 WARN Utils: Your hostname, alex-Inspiron-5566 resolves to a loopback address: 127.0.1.1; using 192.168.18.10 instead (on interface enp3s0)
21/11/11 19:51:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/alex/Dev/pmd/venv/lib/python3.9/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/alex/.ivy2/cache
The jars for the packages stored in: /home/alex/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9714d291-4da0-48e9-8d07-ae37ff80f13d;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 359ms :: artifacts dl 14ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |

In [3]:
## Corrigir gambiarra depois

# # Separando os items do JSON
# for pais in lista_paises:
#     with open('archive/'+ pais +'_category_id.json') as categories:
#         obj = json.load(categories)
#         items = obj['items']

#         with open('archive/'+ pais +'_category_id.json', 'w') as f:
#             json.dump(items, f)

In [3]:
lista_paises = ['CA', 'DE', 'FR', 'GB', 'IN', 'JP', 'KR', 'MX', 'RU', 'US']
base_name_csv = 'videos.csv'
base_name_json = '_category_id.json'
path = 'archive/'
videos = {}
categorias = {}
todos = None

In [4]:
colunas = ['video_id', 'trending_date', 'title', 'category_id', 'publish_time', 'views', 'likes', 'dislikes', \
           'comment_count']

## Lendo os arquivos de vídeos

In [5]:
# Comando de leitura

for pais in lista_paises:
    categorias[pais] = spark.read.format('org.apache.spark.sql.json').option("multiline","true") \
        .load(path+pais+base_name_json)\
        .select("id", f.col("snippet.title").alias('category_title'))
    
    videos[pais] = spark.read.format("csv").option("header", "true").load(path+pais+base_name_csv) \
        .select(colunas) \
        .join(categorias[pais], f.col('category_id') == f.col('id')) \
        .drop('category_id') \
        .drop('id')
    

## Resultados Leitura

In [6]:
## Mostra o schema do Dataframe
videos['US'].printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- category_title: string (nullable = true)



In [7]:
categorias['US'].printSchema()

root
 |-- id: string (nullable = true)
 |-- category_title: string (nullable = true)



In [8]:
categorias['US'].select('*').show()

+---+--------------------+
| id|      category_title|
+---+--------------------+
|  1|    Film & Animation|
|  2|    Autos & Vehicles|
| 10|               Music|
| 15|      Pets & Animals|
| 17|              Sports|
| 18|        Short Movies|
| 19|     Travel & Events|
| 20|              Gaming|
| 21|       Videoblogging|
| 22|      People & Blogs|
| 23|              Comedy|
| 24|       Entertainment|
| 25|     News & Politics|
| 26|       Howto & Style|
| 27|           Education|
| 28|Science & Technology|
| 29|Nonprofits & Acti...|
| 30|              Movies|
| 31|     Anime/Animation|
| 32|    Action/Adventure|
+---+--------------------+
only showing top 20 rows



### 3. Unindo Dataframes

In [9]:
schema = videos['US'].schema
df_videos = spark.createDataFrame([], schema)

In [10]:
df_videos.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- category_title: string (nullable = true)



In [12]:
for pais in lista_paises:
    df_videos = df_videos.union(videos[pais])

In [13]:
df_videos = df_videos.withColumn('views', f.col('views').cast('int')) \
                     .withColumn('likes', f.col('likes').cast('int')) \
                     .withColumn('dislikes', f.col('dislikes').cast('int')) \
                     .withColumn('comment_count', f.col('comment_count').cast('int')) \
                     .withColumn('trending_date', f.to_timestamp(f.col('trending_date'), 'yy.dd.MM')) \
                     .withColumn('publish_time', f.regexp_replace('publish_time', 'T', ' ')) \
                     .withColumn('publish_time', f.regexp_replace('publish_time', 'Z', '')) \
                     .withColumn('publish_time', f.to_timestamp(f.col('publish_time'), 'yyyy-MM-dd HH:mm:ss.SSS'))\
                     .withColumn("trending_date", f.col("trending_date") + f.expr("INTERVAL 1 days"))
    

### 4. Lidando com valores nulos

In [14]:
df_videos = df_videos.na.fill(value=0,subset=['comment_count', 'likes', 'dislikes'])

### 5. Criando coluna interations

In [15]:
df_videos = df_videos.withColumn('interation', f.col('likes')+f.col('dislikes')+f.col('comment_count'))

In [16]:
df_videos.select('video_id', 'comment_count', 'likes', 'dislikes', 'interation').show(truncate=False)

+-----------+-------------+-------+--------+----------+
|video_id   |comment_count|likes  |dislikes|interation|
+-----------+-------------+-------+--------+----------+
|n1WpP7iowLc|125882       |787425 |43420   |956727    |
|0dBIkQ4Mz1M|13030        |127794 |1688    |142512    |
|5qpjK5DgCt4|8181         |146035 |5339    |159555    |
|d380meD0W0M|17518        |132239 |1989    |151746    |
|2Vv-BfVoq4g|85067        |1634130|21082   |1740279   |
|0yIWz1XEeyc|12143        |103755 |4613    |120511    |
|_uM5kFfkhB8|26629        |187464 |9850    |223943    |
|2kyS6SvSYSE|15959        |57534  |2967    |76460     |
|JzCsM1vtn78|36391        |292837 |4123    |333351    |
|43sm-QwLcx4|1484         |4135   |976     |6595      |
|H1KBHFXm2Bg|28976        |263596 |8585    |301157    |
|U3xLOo-CNwo|3312         |28451  |2285    |34048     |
|FyZMnhUtLfE|186          |218    |30      |434       |
|7MxiQ4v0EnE|8063         |127477 |7134    |142674    |
|LUzsOyWp9lw|5335         |31810  |668     |3781

### 6. Criando coluna time_to_trends

In [17]:
df_videos = df_videos.withColumn('time_to_trends', f.col('trending_date').cast('long') - f.col('publish_time').cast('long'))

In [18]:
df_videos.select('video_id', 'trending_date', 'views', 'category_title', 'publish_time', 'time_to_trends').show(truncate=False)

+-----------+-------------------+--------+----------------+-------------------+--------------+
|video_id   |trending_date      |views   |category_title  |publish_time       |time_to_trends|
+-----------+-------------------+--------+----------------+-------------------+--------------+
|n1WpP7iowLc|2017-11-15 00:00:00|17158579|Music           |2017-11-10 17:00:03|370797        |
|0dBIkQ4Mz1M|2017-11-15 00:00:00|1014651 |Comedy          |2017-11-13 17:00:00|111600        |
|5qpjK5DgCt4|2017-11-15 00:00:00|3191434 |Comedy          |2017-11-12 19:05:24|190476        |
|d380meD0W0M|2017-11-15 00:00:00|2095828 |Entertainment   |2017-11-12 18:01:41|194299        |
|2Vv-BfVoq4g|2017-11-15 00:00:00|33523622|Music           |2017-11-09 11:04:14|478546        |
|0yIWz1XEeyc|2017-11-15 00:00:00|1309699 |News & Politics |2017-11-13 07:37:51|145329        |
|_uM5kFfkhB8|2017-11-15 00:00:00|2987945 |Comedy          |2017-11-12 23:52:13|173267        |
|2kyS6SvSYSE|2017-11-15 00:00:00|748374  |People &

In [20]:
df_videos.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: timestamp (nullable = true)
 |-- title: string (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- category_title: string (nullable = true)
 |-- interation: integer (nullable = true)
 |-- time_to_trends: long (nullable = true)



### 7. Criando Dataframe de Categoria

In [21]:

categorias_text = df_videos.select('category_title').distinct().collect()
ids = [(idx+1, row.category_title) for idx, row in enumerate(categorias_text)]
ids

[(1, 'Shows'),
 (2, 'Education'),
 (3, 'Gaming'),
 (4, 'Entertainment'),
 (5, 'Travel & Events'),
 (6, 'Science & Technology'),
 (7, 'Sports'),
 (8, 'Howto & Style'),
 (9, 'Nonprofits & Activism'),
 (10, 'Film & Animation'),
 (11, 'People & Blogs'),
 (12, 'News & Politics'),
 (13, 'Pets & Animals'),
 (14, 'Movies'),
 (15, 'Autos & Vehicles'),
 (16, 'Music'),
 (17, 'Comedy'),
 (18, 'Trailers')]

In [22]:
df_categorias = spark.createDataFrame(ids).toDF('id', 'category_name')


In [23]:
df_categorias.show()

+---+--------------------+
| id|       category_name|
+---+--------------------+
|  1|               Shows|
|  2|           Education|
|  3|              Gaming|
|  4|       Entertainment|
|  5|     Travel & Events|
|  6|Science & Technology|
|  7|              Sports|
|  8|       Howto & Style|
|  9|Nonprofits & Acti...|
| 10|    Film & Animation|
| 11|      People & Blogs|
| 12|     News & Politics|
| 13|      Pets & Animals|
| 14|              Movies|
| 15|    Autos & Vehicles|
| 16|               Music|
| 17|              Comedy|
| 18|            Trailers|
+---+--------------------+



### 8. Atualizando ID das categorias no DataFrame de Videos

In [24]:
df_videos = df_videos.join(df_categorias, f.col('category_title') == f.col('category_name')) \
                    .withColumnRenamed('id', 'category_id') \
                    .drop('category_title')

In [25]:
df_videos.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: timestamp (nullable = true)
 |-- title: string (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- interation: integer (nullable = true)
 |-- time_to_trends: long (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_name: string (nullable = true)



### 9. Criar coluna qtd_view no DataFrame de Categorias

In [32]:
df_max_views = df_videos.groupBy('video_id', 'category_id').agg(f.max('views').alias('max_views'))

df_max_views.show()

+-----------+-----------+---------+
|   video_id|category_id|max_views|
+-----------+-----------+---------+
|AlrbKKGIcjI|          1|   591632|
|syiEqRf0Xp8|          1|   584519|
|FH0NCnqKwmM|          1|   778339|
|7ps84kSkLvQ|          1|    71776|
|rle8kRdPvH0|          1|   101183|
|wUJqPb9Uc34|          1|   179314|
|4kfas12xWOw|          2|     3648|
|W6bUF2VirhI|          2|     9086|
|yfObh93krmE|          2|     3173|
|oVcF6Q0PV4M|          2|    28888|
|2FnDSWqkEAo|          2|    30113|
|Ukjn7A9L7vY|          2|   150609|
|_Ct7yN_iELM|          2|    25120|
|8zQ8PXLxGt4|          2|    34935|
|C7cqCL9BfaE|          2|    54557|
|oycYpm5jCYA|          2|    43150|
|fyzDgCxD8DY|          2|    17147|
|OqZOZ6PViIM|          2|     6681|
|pQWrqUHveAw|          2|     7437|
|gJNkBOcPjLQ|          2|    42501|
+-----------+-----------+---------+
only showing top 20 rows



In [33]:
df_categorias = df_categorias.join( \
                        df_max_views.groupBy('category_id').agg(f.sum('max_views').alias('qtd_views')), \
                        f.col('id') == f.col('category_id')) \
                    .drop('category_id')         

In [34]:
df_categorias.show()

+---+--------------------+-----------+
| id|       category_name|  qtd_views|
+---+--------------------+-----------+
|  7|              Sports| 4053394648|
|  6|Science & Technology| 1276760759|
|  9|Nonprofits & Acti...|   31064609|
| 17|              Comedy| 4585317173|
|  5|     Travel & Events|  180624687|
|  1|               Shows|  151690576|
| 10|    Film & Animation| 3198908495|
|  3|              Gaming| 1424909053|
| 12|     News & Politics| 3478436055|
|  8|       Howto & Style| 2217639651|
| 11|      People & Blogs| 5633249302|
|  2|           Education|  590198575|
|  4|       Entertainment|17643070511|
| 13|      Pets & Animals|  362257052|
| 18|            Trailers|      38453|
| 14|              Movies|    8053865|
| 15|    Autos & Vehicles|  662117761|
| 16|               Music|18232006171|
+---+--------------------+-----------+



In [ ]:
df_categorias.printSchema()

## Salvando dados no MongoDB

In [ ]:
df_categorias.write.format("mongo").mode("overwrite").option("database","teste")\
            .option("collection", "categorias").save()

In [ ]:
df_videos.write.format("mongo").mode("overwrite").option("database","teste") \ 
            .option("collection", "videos").save()

In [19]:
df_videos.select('video_id', 'time_to_trends', 'publish_time', 'trending_date')\
.sort(df_videos.time_to_trends, ascending=True).show()

+-----------+--------------+-------------------+-------------------+
|   video_id|time_to_trends|       publish_time|      trending_date|
+-----------+--------------+-------------------+-------------------+
|KyoL63wIARs|         11300|2018-05-31 20:51:40|2018-06-01 00:00:00|
|KyoL63wIARs|         11300|2018-05-31 20:51:40|2018-06-01 00:00:00|
|dZ0O45CLwlM|         11365|2018-05-31 20:50:35|2018-06-01 00:00:00|
|dZ0O45CLwlM|         11365|2018-05-31 20:50:35|2018-06-01 00:00:00|
|g0je4ji8o1M|         15551|2018-05-31 19:40:49|2018-06-01 00:00:00|
|g0je4ji8o1M|         15551|2018-05-31 19:40:49|2018-06-01 00:00:00|
|ZqWRDbqTUPA|         19706|2018-06-10 18:31:34|2018-06-11 00:00:00|
|ZqWRDbqTUPA|         19706|2018-06-10 18:31:34|2018-06-11 00:00:00|
|kmjJ9K2jP90|         20480|2018-05-31 18:18:40|2018-06-01 00:00:00|
|kmjJ9K2jP90|         20480|2018-05-31 18:18:40|2018-06-01 00:00:00|
|ynEN3LqUuZQ|         21467|2018-06-10 18:02:13|2018-06-11 00:00:00|
|ynEN3LqUuZQ|         21467|2018-0